<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AgenticAI_DrugDiscovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://python.langchain.com/docs/integrations/chat/google_generative_ai/

deepseek

https://platform.deepseek.com/usage

https://api-docs.deepseek.com/quick_start/pricing/

In [ ]:
!pip install -q crewai crewai[tools] -q

In [ ]:
!pip install -q colab-env litellm -q

## test

In [3]:
from google.colab import userdata
from litellm import completion
import os

deepseek_api_key_test = userdata.get('DEEPSEEK_API_KEY')
#print(f"Test Key: {deepseek_api_key_test}")

try:
    response = completion(
        model="deepseek/deepseek-reasoner",
        messages=[{"role": "user", "content": "hello"}],
        api_key=deepseek_api_key_test # Explicitly pass the key
    )
    print("Test call successful!")
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Test call failed: {e}")

Test call successful!
Hello!! 😊 How can I help you today?


## Drug Discover Agent

In [4]:
# Import necessary classes from CrewAI
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool

# Import litellm for the LLM
from litellm import completion

# Import userdata for accessing keys
from google.colab import userdata

# --- Configuration ---
import os

# Set the Deepseek API key as an environment variable for litellm
# This is one way litellm picks up keys.
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
os.environ["DEEPSEEK_API_KEY"] = deepseek_api_key


# --- Define the Disease for Discovery ---
DISEASE = "Alzheimer's Disease" # You can change this to any disease

print(f"Starting Agentic AI Drug Discovery for: {DISEASE}\n")

# --- Define a Custom Tool for Drug Discovery Actions ---
# This tool encapsulates the core action of querying the LLM for drug discovery related information.
# The tool description is now simplified as it doesn't directly instantiate an LLM,
# but rather the _run method can use the CrewAI agent's LLM contextually or call litellm directly.
# However, for complex reasoning within the tool, it's better to have the tool *access* the agent's LLM.
# CrewAI tools are passed the agent's execution context, which includes the LLM.

class DrugDiscoveryTool(BaseTool):
    name: str = "Drug Discovery Research Tool"
    description: str = (
        "A tool capable of performing research, generating ideas, "
        "and providing concise summaries related to drug discovery concepts, "
        "including biological targets, molecule generation, efficacy, and toxicity predictions."
    )

    def _run(self, query: str) -> str:
        """
        Executes the drug discovery related query using the Agent's LLM.
        """
        try:
            response = completion(
                model="deepseek/deepseek-reasoner", # Explicitly use the deepseek model
                messages=[{"role": "user", "content": query}],
                # api_key=os.getenv("DEEPSEEK_API_KEY") # Can explicitly pass, but env var works
            )
            # litellm response format
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error in DrugDiscoveryTool: {e}")
            return f"Error performing research: {e}"

# Instantiate the custom tool - no LLM needed in constructor now
drug_discovery_tool = DrugDiscoveryTool()

# --- Define Agents ---
# Define the model string to be used by agents and the crew
MODEL_NAME = "deepseek/deepseek-reasoner"

# Agent 1: Target Identification Agent
target_identification_agent = Agent(
    role='Target Identification Agent',
    goal=f'Identify 2-3 most promising biological targets for therapeutic intervention against {DISEASE}.',
    backstory=(
        "You are a leading computational biologist specializing in identifying disease-specific molecular targets. "
        "Your expertise lies in analyzing vast biological datasets and pinpointing key proteins or pathways "
        f"that, when modulated, could treat {DISEASE}."
    ),
    verbose=False,
    allow_delegation=False,
    llm=MODEL_NAME, # Agent's internal LLM, uses the model string
    tools=[drug_discovery_tool] # Agent can use this tool for its tasks
)

# Agent 2: Molecule Generation Agent
molecule_generation_agent = Agent(
    role='Molecule Generation Agent',
    goal='Design 2-3 hypothetical drug-like small molecules or compound classes that could interact with the identified biological targets.',
    backstory=(
        "You are an innovative medicinal chemist with a knack for de novo drug design. "
        "You translate biological target information into novel chemical structures, focusing on "
        "drug-like properties and potential for binding."
    ),
    verbose=False,
    allow_delegation=False,
    llm=MODEL_NAME, # Agent's internal LLM
    tools=[drug_discovery_tool] # Agent can use this tool for its tasks
)

# Agent 3: Efficacy Prediction Agent
efficacy_prediction_agent = Agent(
    role='Efficacy Prediction Agent',
    goal='Predict the potential efficacy of the designed drug molecules against the target for the given disease.',
    backstory=(
        "You are a seasoned pharmacologist specializing in in silico efficacy modeling. "
        "You can assess how well a hypothetical drug might perform in a biological system, "
        "based on its proposed mechanism of action and target interaction."
    ),
    verbose=False,
    allow_delegation=False,
    llm=MODEL_NAME, # Agent's internal LLM
    tools=[drug_discovery_tool] # Agent can use this tool for its tasks
)

# Agent 4: Toxicity Prediction Agent
toxicity_prediction_agent = Agent(
    role='Toxicity Prediction Agent',
    goal='Predict the potential toxicity profile and common side effects of the designed drug molecules.',
    backstory=(
        "You are an expert toxicologist with extensive knowledge of computational toxicology. "
        "You can foresee potential adverse effects and safety concerns for novel compounds, "
        "crucial for early drug development."
    ),
    verbose=False,
    allow_delegation=False,
    llm=MODEL_NAME, # Agent's internal LLM
    tools=[drug_discovery_tool] # Agent can use this tool for its tasks
)

# --- Define Tasks ---

# Task 1: Identify Targets
identify_targets_task = Task(
    description=(
        f"Use the 'Drug Discovery Research Tool' to analyze existing knowledge and scientific literature "
        f"related to {DISEASE}. Identify and list 2-3 specific biological targets (e.g., enzymes, receptors, "
        "signaling pathways) that are most relevant for therapeutic intervention. "
        "The output should be a numbered list of targets, briefly explaining why each is a good candidate."
        "\n\nTool Use Example: `Drug Discovery Research Tool.run(\"List 2-3 potential biological targets for Alzheimer's Disease and why.\")`"
    ),
    expected_output=f"A numbered list of 2-3 potential biological targets for {DISEASE} with brief justifications.",
    agent=target_identification_agent,
)

# Task 2: Generate Molecules
generate_molecules_task = Task(
    description=(
        "Based on the identified biological targets from the previous step, "
        "use the 'Drug Discovery Research Tool' to propose 2-3 hypothetical drug-like small molecules or compound classes. "
        "For each target, suggest one or two molecules/compound classes. "
        "Provide very brief, descriptive names (e.g., 'Target-X inhibitor', 'Pathway-Y modulator'). "
        "The output should be a numbered list, linking each molecule/class to its target."
        "\n\nTool Use Example: `Drug Discovery Research Tool.run(\"Suggest 2-3 hypothetical drug molecules for [target from previous step].\")`"
    ),
    expected_output="A numbered list of 2-3 hypothetical drug molecules/compound classes linked to their targets.",
    agent=molecule_generation_agent,
    context=[identify_targets_task] # This task depends on the output of the previous task
)

# Task 3: Predict Efficacy
predict_efficacy_task = Task(
    description=(
        "For each of the hypothetical drug molecules generated in the previous step, "
        "use the 'Drug Discovery Research Tool' to predict its potential efficacy against the respective "
        f"biological target in the context of {DISEASE}. For each molecule, state its predicted efficacy "
        "(e.g., 'High', 'Moderate', 'Low') and provide a concise reason for the prediction, based on its "
        "proposed interaction with the target. The output should be a numbered list, summarizing efficacy for each molecule."
        "\n\nTool Use Example: `Drug Discovery Research Tool.run(\"Predict efficacy for [molecule] targeting [target] for [disease].\")`"
    ),
    expected_output="A numbered list of drug molecules with their predicted efficacy (High/Moderate/Low) and reasons.",
    agent=efficacy_prediction_agent,
    context=[generate_molecules_task] # This task depends on the output of the previous task
)

# Task 4: Predict Toxicity
predict_toxicity_task = Task(
    description=(
        "For each of the hypothetical drug molecules, use the 'Drug Discovery Research Tool' to predict "
        "its potential toxicity profile. State its predicted toxicity level (e.g., 'Low', 'Moderate', 'High') "
        "and list 2-3 possible common side effects or safety concerns. "
        "The output should be a numbered list, summarizing toxicity for each molecule."
        "\n\nTool Use Example: `Drug Discovery Research Tool.run(\"Predict toxicity and side effects for [molecule].\")`"
    ),
    expected_output="A numbered list of drug molecules with their predicted toxicity (Low/Moderate/High) and side effects.",
    agent=toxicity_prediction_agent,
    context=[generate_molecules_task] # This task also depends on the generated molecules
)

# --- Form the Crew ---
# The process is sequential, meaning tasks run in the order they are defined.
drug_discovery_crew = Crew(
    agents=[
        target_identification_agent,
        molecule_generation_agent,
        efficacy_prediction_agent,
        toxicity_prediction_agent
    ],
    tasks=[
        identify_targets_task,
        generate_molecules_task,
        predict_efficacy_task,
        predict_toxicity_task
    ],
    verbose=False, # Increased verbosity for detailed output
    process=Process.sequential, # Tasks run in sequence
    full_output=True, # Get full output including intermediate steps
    llm=MODEL_NAME # The crew also needs an LLM for its overall orchestration and reasoning
)

# --- Execute the Crew ---
print("Running the Drug Discovery Crew...\n")
crew_result = drug_discovery_crew.kickoff()

print("\n--- Drug Discovery Process Completed ---")
if isinstance(crew_result, dict) and 'final_output' in crew_result:
    print("Final Summary of Drug Discovery:")
    print(crew_result['final_output'])
else:
    # If not a dict or 'final_output' key is missing, print the raw result
    print("Raw Crew Result:")
    print(crew_result)


# You can also inspect intermediate results if needed, e.g.:
print('\n')
print("\n--- Intermediate Results ---")
# Check if the tasks have an output attribute and print it
print("Targets identified:", getattr(identify_targets_task, 'output', 'No output'))
print("Molecules generated:", getattr(generate_molecules_task, 'output', 'No output'))
print("Efficacy predictions:", getattr(predict_efficacy_task, 'output', 'No output'))
print("Toxicity predictions:", getattr(predict_toxicity_task, 'output', 'No output'))

Starting Agentic AI Drug Discovery for: Alzheimer's Disease

Running the Drug Discovery Crew...


--- Drug Discovery Process Completed ---
Raw Crew Result:
1. **Amyloid-beta (Aβ) Core-Solubilizing Disruptor (diphenyloxazole core + sulfonated dendrimer tail)**  
   - **Predicted Toxicity**: High  
   - **Common Side Effects**: Gastrointestinal disturbances (nausea, vomiting, diarrhea), Neurological symptoms (headache, dizziness), Injection-site reactions (pain, swelling)  
   - *Key Safety Concerns*: Hepatotoxicity, ARIA-like neurovascular events, and immunogenicity due to dendrimer component.

2. **Tau Phospho-Pocket Chaperone (benzothiazole-glycoconjugate)**  
   - **Predicted Toxicity**: High  
   - **Common Side Effects**: Gastrointestinal distress (nausea, vomiting), Metabolic disruption (hyperglycemia), Neurological effects (headache, dizziness)  
   - *Key Safety Concerns*: Hepatotoxicity from benzothiazole metabolism, cardiotoxicity (QT prolongation), and potential kidney injury